In [1]:
import yfinance as yf
import pandas as pd
from initial_functions import fn_read_from_db

========| ('irfan_admin', 'trade_app') |========
*** ✅ SUCCESSFUL CLOUD CONNECTION ⛓️ ***


In [ ]:
df_raw = fn_read_from_db('tbl_ticker_actual_daily_2')

In [127]:
TICKER = 'CIMSA.IS'
df_t = df_raw[df_raw['TICKER']==TICKER]

In [129]:
def get_highest_high(df):
    # 'HIGH' sütunundaki en büyük değerin indeksini bulur
    max_idx = df['HIGH'].idxmax()
    
    # İlgili indeksteki tarih ve fiyat değerlerini çeker
    high_date = df.loc[max_idx, 'DATETIME']
    high_value = df.loc[max_idx, 'HIGH']
    
    return high_date, high_value

# Örnek Kullanım:
date, value = get_highest_high(df_t)
print(f"Tarih: {date}, En Yüksek Değer: {value}")

Tarih: 2025-01-28 00:00:00+03:00, En Yüksek Değer: 56.69278419521964


In [131]:
def filter_from_high_to_end(df):
    # 1. DATETIME sütununu datetime objesine çevir (garantiye almak için)
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])
    
    # 2. Tarihe göre küçükten büyüğe sırala
    df = df.sort_values('DATETIME').reset_index(drop=True)
    
    # 3. En yüksek HIGH değerinin olduğu satırı ve tarihi bul
    max_idx = df['HIGH'].idxmax()
    high_date = df.loc[max_idx, 'DATETIME']
    
    # 4. En yüksek değerin olduğu indeksten sonuna kadar filtrele
    # (Sıralı olduğu için bu indeks ve sonrası tüm geleceği kapsar)
    filtered_df = df.iloc[max_idx:].copy()
    
    return filtered_df

# Kullanım:
df_res = filter_from_high_to_end(df_t)

/var/folders/xx/qnwh4gps12301qzdq0t86dq00000gn/T/ipykernel_36908/1929849138.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATETIME'] = pd.to_datetime(df['DATETIME'])


In [133]:
import numpy as np
import pandas as pd


def calculate_tv_frvp(
    df,
    value_area_pct=68,
    row_size=1
):
    """
    TradingView FRVP (Fixed Range Volume Profile) hesaplama
    Parametreler:
        df: 1 dakikalık OHLCV dataframe
            Sütunlar: OPEN, HIGH, LOW, CLOSE, VOLUME
        value_area_pct: Value Area yüzdesi (örn 68)
        row_size: Tick per row (örn 1)

    Döndürür:
        dict -> {POC, VAL, VAH}
    """

    df = df.copy()

    # ---- GLOBAL PRICE GRID ----
    price_min = np.floor(df['LOW'].min() / row_size) * row_size
    price_max = np.ceil(df['HIGH'].max() / row_size) * row_size

    price_levels = np.arange(price_min, price_max + row_size, row_size)
    hist = pd.Series(0.0, index=price_levels)

    # ---- VOLUME DISTRIBUTION (Up/Down mantığı) ----
    for _, row in df.iterrows():

        low = np.floor(row['LOW'] / row_size) * row_size
        high = np.ceil(row['HIGH'] / row_size) * row_size

        levels = np.arange(low, high + row_size, row_size)

        if len(levels) == 0:
            continue

        # Up / Down volume ayrımı
        volume = row['VOLUME']

        # Hacmi ticklere eşit dağıt (TV mantığına en yakın)
        vol_per_level = volume / len(levels)

        hist.loc[levels] += vol_per_level

    # ---- POC ----
    max_volume = hist.max()

    # Birden fazla max varsa → ortadaki (TV davranışı)
    poc_candidates = hist[hist == max_volume].index.values
    poc = poc_candidates[len(poc_candidates) // 2]

    # ---- VALUE AREA ----
    total_volume = hist.sum()
    target_volume = total_volume * (value_area_pct / 100)

    sorted_prices = hist.sort_index().index.values
    poc_pos = np.where(sorted_prices == poc)[0][0]

    included = {poc}
    cum_volume = hist.loc[poc]

    lower = poc_pos - 1
    upper = poc_pos + 1

    while cum_volume < target_volume:

        vol_down = hist.iloc[lower] if lower >= 0 else -1
        vol_up = hist.iloc[upper] if upper < len(hist) else -1

        if vol_up > vol_down:
            cum_volume += vol_up
            included.add(sorted_prices[upper])
            upper += 1

        elif vol_down > vol_up:
            cum_volume += vol_down
            included.add(sorted_prices[lower])
            lower -= 1

        else:
            # Tie-break:
            # 1) POC'a yakın olan
            # 2) Eşitse yukarıyı seç
            if lower >= 0 and upper < len(hist):
                dist_down = abs(sorted_prices[lower] - poc)
                dist_up = abs(sorted_prices[upper] - poc)

                if dist_up <= dist_down:
                    cum_volume += vol_up
                    included.add(sorted_prices[upper])
                    upper += 1
                else:
                    cum_volume += vol_down
                    included.add(sorted_prices[lower])
                    lower -= 1
            elif upper < len(hist):
                cum_volume += vol_up
                included.add(sorted_prices[upper])
                upper += 1
            elif lower >= 0:
                cum_volume += vol_down
                included.add(sorted_prices[lower])
                lower -= 1
            else:
                break

    VAL = min(included)
    VAH = max(included)

    return {
        "POC": float(poc),
        "VAL": float(VAL),
        "VAH": float(VAH)
    }

In [135]:
calculate_tv_frvp(df_res)

{'POC': 46.0, 'VAL': 45.0, 'VAH': 51.0}

## new

In [139]:
import numpy as np
import pandas as pd


def detect_tick_size(df):
    """
    Veriden minimum gerçek tick'i bulur
    """
    prices = pd.concat([df['OPEN'], df['HIGH'], df['LOW'], df['CLOSE']])
    diffs = np.diff(np.sort(prices.unique()))
    diffs = diffs[diffs > 0]
    return np.round(diffs.min(), 6)


def calculate_tv_frvp_v2(
    df,
    value_area_pct=68,
    row_size=1
):
    """
    TradingView FRVP - Tick doğru hesaplanmış versiyon
    """

    df = df.copy()

    # ---- Gerçek tick size tespit ----
    tick = detect_tick_size(df)

    # Row size = tick per row
    price_step = tick * row_size

    # ---- GLOBAL PRICE GRID ----
    price_min = np.floor(df['LOW'].min() / price_step) * price_step
    price_max = np.ceil(df['HIGH'].max() / price_step) * price_step

    price_levels = np.arange(price_min, price_max + price_step, price_step)
    hist = pd.Series(0.0, index=np.round(price_levels, 6))

    # ---- Volume dağıtımı (TV'ye daha yakın lineer model) ----
    for _, row in df.iterrows():

        low = np.floor(row['LOW'] / price_step) * price_step
        high = np.ceil(row['HIGH'] / price_step) * price_step

        levels = np.arange(low, high + price_step, price_step)
        levels = np.round(levels, 6)

        if len(levels) == 0:
            continue

        # Up / Down volume
        volume = row['VOLUME']

        # Lineer ağırlık (close'a doğru hafif artan)
        weights = np.linspace(1, 2, len(levels))
        weights = weights / weights.sum()

        hist.loc[levels] += volume * weights

    # ---- POC ----
    max_volume = hist.max()
    poc_candidates = hist[hist == max_volume].index.values

    # TV genelde ortadaki en stabil olanı seçer
    poc = float(poc_candidates[len(poc_candidates) // 2])

    # ---- VALUE AREA ----
    total_volume = hist.sum()
    target_volume = total_volume * (value_area_pct / 100)

    sorted_prices = hist.sort_index().index.values
    poc_pos = np.where(sorted_prices == poc)[0][0]

    included = {poc}
    cum_volume = hist.loc[poc]

    lower = poc_pos - 1
    upper = poc_pos + 1

    while cum_volume < target_volume:

        vol_down = hist.iloc[lower] if lower >= 0 else -1
        vol_up = hist.iloc[upper] if upper < len(hist) else -1

        if vol_up > vol_down:
            cum_volume += vol_up
            included.add(sorted_prices[upper])
            upper += 1

        elif vol_down > vol_up:
            cum_volume += vol_down
            included.add(sorted_prices[lower])
            lower -= 1

        else:
            if lower >= 0 and upper < len(hist):
                dist_down = abs(sorted_prices[lower] - poc)
                dist_up = abs(sorted_prices[upper] - poc)

                if dist_up <= dist_down:
                    cum_volume += vol_up
                    included.add(sorted_prices[upper])
                    upper += 1
                else:
                    cum_volume += vol_down
                    included.add(sorted_prices[lower])
                    lower -= 1
            elif upper < len(hist):
                cum_volume += vol_up
                included.add(sorted_prices[upper])
                upper += 1
            elif lower >= 0:
                cum_volume += vol_down
                included.add(sorted_prices[lower])
                lower -= 1
            else:
                break

    VAL = float(min(included))
    VAH = float(max(included))

    return {
        "POC": round(poc, 6),
        "VAL": round(VAL, 6),
        "VAH": round(VAH, 6)}


In [141]:
print(TICKER)
print(f"Tarih: {date}, En Yüksek Değer: {value}")

calculate_tv_frvp_v2(df_res)

CIMSA.IS
Tarih: 2025-01-28 00:00:00+03:00, En Yüksek Değer: 56.69278419521964
nan 0.0 nan


/var/folders/xx/qnwh4gps12301qzdq0t86dq00000gn/T/ipykernel_36908/1317233843.py:33: RuntimeWarning: divide by zero encountered in scalar divide
  price_min = np.floor(df['LOW'].min() / price_step) * price_step
/var/folders/xx/qnwh4gps12301qzdq0t86dq00000gn/T/ipykernel_36908/1317233843.py:33: RuntimeWarning: invalid value encountered in scalar multiply
  price_min = np.floor(df['LOW'].min() / price_step) * price_step
/var/folders/xx/qnwh4gps12301qzdq0t86dq00000gn/T/ipykernel_36908/1317233843.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  price_max = np.ceil(df['HIGH'].max() / price_step) * price_step
/var/folders/xx/qnwh4gps12301qzdq0t86dq00000gn/T/ipykernel_36908/1317233843.py:34: RuntimeWarning: invalid value encountered in scalar multiply
  price_max = np.ceil(df['HIGH'].max() / price_step) * price_step


ValueError: arange: cannot compute length